### Import Modules

In [1]:
import os
import requests
from datetime import datetime, timedelta
import pytz
from google import genai
from google.genai import types

### Define and Implement Function 

In [2]:
def get_flight_status(flight: str) -> dict:
    """Returns live flight status information for a given flight ID."""
    AEROAPI_BASE_URL = "https://aeroapi.flightaware.com/aeroapi"
    AEROAPI_KEY = os.getenv("AERO_API_KEY") 

    if not AEROAPI_KEY:
        raise ValueError("AERO_API_KEY is not set in the environment variables.")

    def get_api_session():
        session = requests.Session()
        session.headers.update({"x-apikey": AEROAPI_KEY})
        return session

    def fetch_flight_data(flight_id, session):
        if "flight_id=" in flight_id:
            flight_id = flight_id.split("flight_id=")[1]

        start_date = datetime.now().date().strftime('%Y-%m-%d')
        end_date = (datetime.now().date() + timedelta(days=1)).strftime('%Y-%m-%d')
        api_resource = f"/flights/{flight_id}?start={start_date}&end={end_date}"
        response = session.get(f"{AEROAPI_BASE_URL}{api_resource}")
        response.raise_for_status()
        return response.json()['flights'][0]

    def utc_to_local(utc_date_str, local_timezone_str):
        utc_datetime = datetime.strptime(utc_date_str, '%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=pytz.utc)
        local_timezone = pytz.timezone(local_timezone_str)
        local_datetime = utc_datetime.astimezone(local_timezone)
        return local_datetime.strftime('%Y-%m-%d %H:%M:%S')

    session = get_api_session()
    flight_data = fetch_flight_data(flight, session)

    dep_key = 'estimated_out' if 'estimated_out' in flight_data and flight_data['estimated_out'] else \
              'actual_out' if 'actual_out' in flight_data and flight_data['actual_out'] else \
              'scheduled_out'

    arr_key = 'estimated_in' if 'estimated_in' in flight_data and flight_data['estimated_in'] else \
              'actual_in' if 'actual_in' in flight_data and flight_data['actual_in'] else \
              'scheduled_in'

    flight_details = {
        'flight': flight,
        'source': flight_data['origin']['city'],
        'destination': flight_data['destination']['city'],
        'depart_time': utc_to_local(flight_data[dep_key], flight_data['origin']['timezone']),
        'arrival_time': utc_to_local(flight_data[arr_key], flight_data['destination']['timezone']),
        'status': flight_data['status']
    }
    return flight_details

### Initialize Client

In [3]:
client = genai.Client()

### Wire the Function

In [4]:
config = types.GenerateContentConfig(
    tools=[get_flight_status]
)

### Invoke Gemini API

In [5]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What’s the status of EK226?",
    config=config,
)

### Access the Output

In [6]:
print(response.text)

Flight EK226 from San Francisco to Dubai is currently En Route / On Time. It departed on 2025-07-08 at 17:05:00 and is expected to arrive on 2025-07-09 at 19:12:00.
